In [1]:
print("ok")


ok


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.document_loaders import PyPDFLoader
file_path= "../data/stats.pdf"
loader= PyPDFLoader(file_path)
data= loader.load()

In [9]:
question_gen=""

for page in data:
    question_gen += page.page_content

In [ ]:
question_gen